In [1]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

In [2]:
from pyspark.context import SparkContext 
sc = SparkContext.getOrCreate()
from pyspark.sql.session import SparkSession
#sc = SparkContext.getOrCreate('local')
spark = SparkSession(sc)

In [11]:
import os
os.getcwd()

path = "C:\ML\Spark\spark-2.1.0-bin-hadoop2.6\\"
mpath = "D:\Grad Notes\Search\Project\Data\yelp_dataset\dataset\\"

In [17]:
import pandas as pd

In [19]:
##Toy Dataset code

df_rev = pd.read_csv(mpath+"review.csv")
# lines = spark.read.csv(mpath+"review.csv").rdd
# parts = lines.map(lambda row: row.value.split("::"))

In [20]:
ratings = spark.createDataFrame(df_rev)

KeyboardInterrupt: 

In [ ]:

# ratingsRDD = parts.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]),
#                                      rating=float(p[2]), timestamp=long(p[3])))
# ratings = spark.createDataFrame(ratingsRDD)
(training, test) = ratings.randomSplit([0.8, 0.2])

# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating")
model = als.fit(training)

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

# # Generate top 10 movie recommendations for each user
# userRecs = model.recommendForAllUsers(10)
# # Generate top 10 user recommendations for each movie
# movieRecs = model.recommendForAllItems(10)